# Measuremments with Representative

### Imports

In [1]:
from GraphType import GraphStat
from GraphType import GraphCollection
from scipy import stats
from ipywidgets import interact, fixed, interactive
import readCSV as reader
import ipywidgets as widgets
import matplotlib.pyplot as plt
import random
import numpy as np
import constants


### Classes

* Record the distances of different metrics using a representative

In [2]:
class GraphDistanceWithRep:
    #init with a graph stat and a collection of graph stats
    def __init__(self, graphStat, rep):
        self.graph = graphStat
        self.rep = rep
        self.out_d_distance, _ = stats.ks_2samp(graphStat.out_d, rep.out_d)
        self.na_distance,_ = stats.ks_2samp(graphStat.na, rep.na)
        self.mpc_distance,_ = stats.ks_2samp(graphStat.mpc, rep.mpc)


### Methods


* Find the median ks distance of the same number of nodes

In [3]:
def find_median(x, metric_distances):
    distance_dic = {}
    for index, num_of_nodes in enumerate(x):
        if num_of_nodes[0] not in distance_dic:
            distance_dic[num_of_nodes[0]] = []
        distance_dic[num_of_nodes[0]].append(metric_distances[index])
    median_x = []
    y = []
    for num_of_nodes, distances in distance_dic.items():
        median_x.append(num_of_nodes)
        y.append(np.median(distances))
    order = np.argsort(median_x)
    median_x = np.array(median_x)[order]
    median_y = np.array(y)[order]
    return median_x, median_y


* Plot Diagram

In [20]:
# metric_selector: GraphDistance -> float
def plot(infos, lines, id, metric_selector,colors, title):
    metric_distances = retrive_info_from_list(metric_selector, list(infos.values()))
    x = retrive_info_from_list(lambda a : a.graph.num_nodes, list(infos.values()))
    graph = plt.figure(id,figsize=(18, 10))
    plt.title(title)
    plt.plot(x, metric_distances, color='red', linestyle='', marker='o',alpha=0.7)
    #plot ks distance median
    median_x, median_y = find_median(x, metric_distances)
    plt.plot(median_x, median_y, color='black',marker='o')
    for i in range(0, len(lines)):
        line_infos = retrive_info_from_list(lambda a: infos[a], lines[i])
        line_y = retrive_info_from_list(metric_selector, line_infos)
        line_x = retrive_info_from_list(lambda a : a.graph.num_nodes, line_infos)
        plt.plot(line_x, line_y, marker='o', color=colors[i])
    #graph.show()

* Retrieve information from a list 

In [5]:
def retrive_info_from_list(selector, distances):
    return list(map(selector, distances))

In [6]:
def readStats(path, numModels):
    names =  reader.readmultiplefiles(path, numModels, False)
    stats = []
    for name in names:
        stats.append(GraphStat(name))
    return stats

In [7]:
def calDistanceDic(stats, rep):
    dic = {}
    for info in stats:
        info = GraphDistanceWithRep(info, rep)
        dic[info.graph.id] = info
    return dic

## Read Models

In [8]:
### Read Models
#read representative
human_rep = GraphStat(constants.HUMAN_OUT_D_REP)
human_na = GraphStat(constants.HUMAN_NA_REP)
human_mpc = GraphStat(constants.HUMAN_MPC_REP)

# assign rep distributions to human_rep
human_rep.na = human_na.na
human_rep.mpc = human_mpc.mpc
# Read generated models
viatra_no_con_stats = readStats('../statistics/viatra_nocon_output/', 5000)
viatra_con_stats = readStats('../statistics/viatra_con_output/',5000)
random_stats = readStats('../statistics/random_output/',5000)
real_random_stats = readStats('../statistics/real_random_output/', 10000)
viatra_500_stats = readStats('../statistics/viatra500nodes/', 10000)

## calculate distribution distantces

In [9]:
viatra_no_con_dic = calDistanceDic(viatra_no_con_stats, human_rep)
viatra_con_dic = calDistanceDic(viatra_con_stats, human_rep)
random_dic = calDistanceDic(random_stats, human_rep)
real_random_dic = calDistanceDic(real_random_stats, human_rep)
viatra_500_dic = calDistanceDic(viatra_500_stats, human_rep)

In [30]:
filenames = reader.readmultiplefiles('../statistics/viatra_nocon_output/trajectories/', 15, False)
trajectories = {}
for name in filenames:
    trajectories[name] = reader.readTrajectory(name)

w = widgets.SelectMultiple(
    options = trajectories,
    value = [trajectories[filenames[0]]],
    description='Trajectory:',
    disabled=False,
)

#generate random color for each line
colors = []

for i in range(0, len(trajectories)):
    color = "#%06x" % random.randint(0, 0xFFFFFF)
    colors.append(color)

In [31]:
def plot_out_degree(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(0,), options={'../statistics/viatra_noc…

<function __main__.plot_out_degree(lines)>

In [32]:
def plot_out_na(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity')
interact(plot_out_na, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(11,), options={'../statistics/viatra_no…

<function __main__.plot_out_na(lines)>

In [33]:
def plot_out_mpc(lines):
    plot(viatra_no_con_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC')
interact(plot_out_mpc, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(11,), options={'../statistics/viatra_no…

<function __main__.plot_out_mpc(lines)>

In [35]:
filenames = reader.readmultiplefiles('../statistics/viatra_con_output/trajectories/', 15, False)
trajectories = {}
for name in filenames:
    trajectories[name] = reader.readTrajectory(name)

w = widgets.SelectMultiple(
    options = trajectories,
    value = [trajectories[filenames[0]]],
    description='Trajectory:',
    disabled=False,
)

#generate random color for each line
colors = []

for i in range(0, len(trajectories)):
    color = "#%06x" % random.randint(0, 0xFFFFFF)
    colors.append(color)

In [36]:
def plot_out_degree(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(0,), options={'../statistics/viatra_con…

<function __main__.plot_out_degree(lines)>

In [37]:
def plot_na(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity')
interact(plot_na, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(3,), options={'../statistics/viatra_con…

<function __main__.plot_na(lines)>

In [38]:
def plot_mpc(lines):
    plot(viatra_con_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC')
interact(plot_mpc, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(3,), options={'../statistics/viatra_con…

<function __main__.plot_mpc(lines)>

In [29]:
def plot_out_degree(lines):
    plot(random_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [28]:
def plot_out_degree(lines):
    plot(random_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [26]:
def plot_out_degree(lines):
    plot(random_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [25]:
def plot_out_degree(lines):
    plot(real_random_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [24]:
def plot_out_degree(lines):
    plot(real_random_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [27]:
def plot_out_degree(lines):
    plot(real_random_dic, lines, 0, lambda a: a.mpc_distance, colors, 'MPC')
interact(plot_out_degree, lines=[[]])

interactive(children=(Dropdown(description='lines', options=([],), value=[]), Output()), _dom_classes=('widget…

<function __main__.plot_out_degree(lines)>

In [11]:
filenames = reader.readmultiplefiles('../statistics/viatra500nodes/trajectories/', 15, False)
trajectories = {}
for name in filenames:
    trajectories[name] = reader.readTrajectory(name)

w = widgets.SelectMultiple(
    options = trajectories,
    value = [trajectories[filenames[0]]],
    description='Trajectory:',
    disabled=False,
)

#generate random color for each line
colors = []

for i in range(0, len(trajectories)):
    color = "#%06x" % random.randint(0, 0xFFFFFF)
    colors.append(color)

In [12]:
def plot_out_degree(lines):
    plot(viatra_500_dic, lines, 0, lambda a: a.na_distance, colors, 'node activity')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(0,), options={'../statistics/viatra500n…

<function __main__.plot_out_degree(lines)>

In [22]:
def plot_out_degree(lines):
    plot(viatra_500_dic, lines, 0, lambda a: a.out_d_distance, colors, 'out degree')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(0,), options={'../statistics/viatra500n…

<function __main__.plot_out_degree(lines)>

In [21]:
def plot_out_degree(lines):
    plot(viatra_500_dic, lines, 0, lambda a: a.mpc_distance, colors, 'mpc')
interact(plot_out_degree, lines=w)

interactive(children=(SelectMultiple(description='Trajectory:', index=(0,), options={'../statistics/viatra500n…

<function __main__.plot_out_degree(lines)>